In [34]:
import copy
import random
from collections import defaultdict
import pandas as pd
from src import DATA_DIR

In [35]:
class Resource:
    def __init__(self, name, capacity):
        self.name = name
        self.capacity = capacity
        self.available = capacity
        self.free_up_resources = defaultdict(int)



class Variant:
    def __init__(self, name, activities):
        self.name = name
        self.activities = activities



class Trace:
    def __init__(self, variant, entry_time=0):
        self.variant_name = variant.name
        self.activities = variant.activities
        self.todo_activities = copy.deepcopy(variant.activities)
        self.doing_activities = []  # (<activity>, <start time>, <end time>)
        self.done_activities = []
        self.entry_time = entry_time
        self.start_time = None
        self.end_time = None



class Activity:
    def __init__(self, name, duration, resources):
        self.name = name
        self.duration = duration
        self.resources = resources



class Simulator:
    def __init__(self, traces, resources):
        self.current_time = 0
        self.traces = []
        for variant, details in traces:
            for ind in range(details['Count']):
                self.traces.append(Trace(variant, details['Entry_Time']))
        
        self.resources = dict()
        for item in resources:
            self.resources[item.name] = item


    def update_trace_doing_activities_state(self, trace):
        for ind, doing_item in enumerate(trace.doing_activities):
            if doing_item[2] == self.current_time:
                done_activity = trace.doing_activities.pop(ind)
                trace.done_activities.append(done_activity)
                if len(trace.todo_activities) == 0 and len(trace.doing_activities) == 0:
                    trace.end_time = self.current_time
                    
    
    def free_up_resources(self):
        for resource_name, resource in self.resources.items():
            if self.current_time in self.resources[resource_name].free_up_resources.keys():
                self.resources[resource_name].available += self.resources[resource_name].free_up_resources[self.current_time]
                self.resources[resource_name].free_up_resources.pop(self.current_time)
                


    def allocate_resources(self, activity):
        # Check if resources are available
        flag = True
        for resource_name, resource_required in activity.resources.items():
            resource = self.resources[resource_name]
            if resource.available < resource_required:
                flag = False
                break
        if not flag:
            return "Resourses are not available"

        # Allocate resources
        for resource_name, resource_required in activity.resources.items():
            self.resources[resource_name].available -= resource_required
            self.resources[resource_name].free_up_resources[self.current_time + activity.duration] += resource_required


    def run(self, duration):
        while self.current_time < duration:
            for trace in self.traces:
                # Update trace state doing state
                self.update_trace_doing_activities_state(trace)

                # Pass done traces and the traces that their start data is bigger than current time                
                if len(trace.todo_activities) == 0 or len(trace.doing_activities) > 0 or trace.entry_time > self.current_time:
                    continue

                # Select first activity of trace
                activity = trace.todo_activities[0]

                # Allocate resources
                message = self.allocate_resources(activity)
                if message == "Resourses are not available":
                    continue

                # Assign Trace Start Time
                if len(trace.doing_activities) == 0 and len(trace.done_activities) == 0:
                    trace.start_time = self.current_time

                # Moving activity from todo to doing
                doing_activity = trace.todo_activities.pop(0)
                trace.doing_activities.append((doing_activity, self.current_time, self.current_time + activity.duration))
            
            # Free up resources
            self.free_up_resources()

            self.current_time += 1


    def report_traces(self):
        trace_info_list = []

        for trace in self.traces:
            sum_of_activity = sum(item.duration for item in trace.activities)    
            duration = None
            waiting_time = None

            if trace.end_time:
                duration = trace.end_time
                waiting_time = duration - sum_of_activity

            trace_info_list.append([trace.variant_name,
                                    trace.start_time,
                                    trace.end_time,
                                    sum_of_activity,
                                    duration,
                                    waiting_time])

        return pd.DataFrame(trace_info_list, columns=['Variant', 
                                                      'CaseStartTime',
                                                      'CaseEndTime',
                                                      'TotalDurationOfActivity',
                                                      'CaseTotalDuration',
                                                      'WaitingTime'])

In [53]:
# Define resources
resource1 = Resource("Resource 1", 10)
resource2 = Resource("Resource 2", 10)
resource3 = Resource("Resource 3", 10)
resources = {resource1, resource2, resource3}


# Define activities
activity1 = Activity("Activity 1", 5, {"Resource 1": 1})
activity2 = Activity("Activity 2", 9, {"Resource 1": 1, "Resource 2": 2})
activity3 = Activity("Activity 3", 7, {"Resource 2": 1})
activity4 = Activity("Activity 4", 3, {"Resource 3": 1})


# Define Variants
variant1 = Variant("Variant 1", [activity1, activity2, activity3])
variant2 = Variant("Variant 2", [activity3, activity2, activity4])
variant3 = Variant("Variant 3", [activity4, activity1, activity3])


# Define Traces
traces = [(variant1, {'Count': 10, 'Entry_Time': 0}),
          (variant1, {'Count': 10, 'Entry_Time': 100}),
          (variant2, {'Count': 20, 'Entry_Time': 10}),
          (variant3, {'Count': 15, 'Entry_Time': 5})
         ]


# Create and run the simulator
simulator = Simulator(traces, resources)
simulator.run(duration=1000)

In [54]:
simulator.report_traces()

,Variant,CaseStartTime,CaseEndTime,TotalDurationOfActivity,CaseTotalDuration,WaitingTime
0,Variant 1,0,23,21,23,2
1,Variant 1,0,23,21,23,2
2,Variant 1,0,23,21,23,2
3,Variant 1,0,23,21,23,2
4,Variant 1,0,23,21,23,2
5,Variant 1,0,33,21,33,12
6,Variant 1,0,33,21,33,12
7,Variant 1,0,41,21,41,20
8,Variant 1,0,41,21,41,20
9,Variant 1,0,41,21,41,20


In [55]:
simulator.report_traces().WaitingTime.mean()

63.345454545454544

In [56]:
# AS-IS: 387.6

# TO-BE:

    # Sen.1:   resourse 1: 10      387.30
    # Sen.2:   resourse 2: 10      356.58
    # Sen.3:   resourse 3: 10      63
    # Sen.4:   all resourses: 10  

In [ ]:
class Student:
    def __init__(self, age, gender, city):
        self.age = age
        self.gender = gender
        self.city = city

    def Calculate_Grade():
        pass

In [ ]:
student1 = Student(age=20, gender='Male', city='Tehran')
student2 = Student(age=18, gender='Female', city='Shiraz')

student2.city

'Shiraz'

In [ ]:
student2.Calculate_Grade  ~   Student.Calculate_Grade(student2)

TypeError: Calculate_Grade() takes 0 positional arguments but 1 was given

In [ ]:
Student.Calculate_Grade()